In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy

In [17]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [18]:
#turn cities into df
df = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [19]:
df.head()

,City,lat,lng
0,brenham,30.070154,-96.109967
1,beeville,28.893784,-97.557720
2,lockhart,29.326321,-97.384802
3,san angelo,31.201086,-99.758061
4,lakeway,30.300185,-98.329762


In [20]:
#check length of cities
len(list(df["City"].unique()))

114

In [21]:
cities = list(df["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['brenham',
 'beeville',
 'lockhart',
 'san+angelo',
 'lakeway',
 'hondo',
 'lufkin',
 'port+arthur',
 'tomball',
 'katy',
 'bryan',
 'humble',
 'taylor',
 'fredericksburg',
 'kerrville',
 'leon+valley',
 'stephenville',
 'clute',
 'temple',
 'atascocita',
 'galveston',
 'natchitoches',
 'waco',
 'rosenberg',
 'sulphur',
 'liberty',
 'copperas+cove',
 'brownwood',
 'spring',
 'harker+heights',
 'lumberton',
 'robinson',
 'the+woodlands',
 'lake+jackson',
 'seguin',
 'victoria',
 'angleton',
 'palestine',
 'pleasanton',
 'leander',
 'round+rock',
 'gatesville',
 'bellmead',
 'conroe',
 'killeen',
 'huntsville',
 'nacogdoches',
 'jacksonville',
 'uvalde',
 'vidor',
 'league+city',
 'san+antonio',
 'belton',
 'bay+city',
 'el+campo',
 'nederland',
 'orange',
 'santa+fe',
 'kirby',
 'georgetown',
 'channelview',
 'mission+bend',
 'missouri+city',
 'baytown',
 'corsicana',
 'abilene',
 'canyon+lake',
 'wharton',
 'port+lavaca',
 'del+rio',
 'freeport',
 'alvin',
 'hewitt',
 'la+marque',
 'p

In [22]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [23]:
unix_time()

[1630213200, 1630126800, 1630040400, 1629954000, 1629867600]

In [67]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},US-TX,US&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
        except:
            print("City not found. Skipping...")
            pass
        for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
            hist_json = requests.get(historical_url).json()
            #temp.append(hist_json["current"]["temp"])
            #pressure.append(hist_json["current"]["pressure"])
            #wind_speed.append(hist_json["current"]["wind_speed"])
            #print(historical_url)
    d=dict()
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)

In [68]:
find_location(target_cities)

{'Temperature': [], 'Wind_Speed': [], 'Pressure': []}

In [ ]:
print(len(temp))
print(len(pressure))
print(len(wind_speed))

In [32]:
dict_data = {"Temp" :temp,
             "Pressure":pressure,
             "Wind Speed":wind_speed
            }
df = pd.DataFrame(dict_data)
df

,Temp,Pressure,Wind Speed
